# Dataset Preprocessing

## Enviroment Setup

In [ ]:
%pip install pandas numpy requests Pillow dotenv

In [2]:
import pandas as pd
import numpy as np
import requests
from PIL import Image
from io import BytesIO
import os
import re
from typing import Optional
import time
from dotenv import load_dotenv

## Visualizing data

In [3]:
movies_train = pd.read_csv('data/movies_train.csv')
movies_test = pd.read_csv('data/movies_test.csv')

print("Movies Train:", movies_train.shape, '\n', movies_train.head(), '\n', movies_train.columns)
print("\nMovies Test:", movies_test.shape, '\n', movies_test.head(), '\n', movies_test.columns)

Movies Train: (6819, 23) 
    movieId                             title          genres  \
0      619                         Ed (1996)          Comedy   
1    33826                Saint Ralph (2004)    Comedy|Drama   
2     1298       Pink Floyd: The Wall (1982)   Drama|Musical   
3   140289              Men & Chicken (2015)    Comedy|Drama   
4     3064  Poison Ivy: New Seduction (1997)  Drama|Thriller   

   (no genres listed)  Action  Adventure  Animation  Children  Comedy  Crime  \
0                   0       0          0          0         0       1      0   
1                   0       0          0          0         0       1      0   
2                   0       0          0          0         0       0      0   
3                   0       0          0          0         0       1      0   
4                   0       0          0          0         0       0      0   

   ...  Film-Noir  Horror  IMAX  Musical  Mystery  Romance  Sci-Fi  Thriller  \
0  ...          0       0  

In [4]:
movielens_movies = pd.read_csv('ml-25m/movies.csv')
movielens_links = pd.read_csv('ml-25m/links.csv')

movielens_links['tmdbId'] = movielens_links['tmdbId'].astype('Int64')

print("Movielens Movies:", movielens_movies.shape, '\n', movielens_movies.head(), '\n', movielens_movies.columns)
print("\nMovielens Links:", movielens_links.shape, '\n', movielens_links.head(), '\n', movielens_links.columns)

Movielens Movies: (62423, 3) 
    movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy   
 Index(['movieId', 'title', 'genres'], dtype='object')

Movielens Links: (62423, 3) 
    movieId  imdbId  tmdbId
0        1  114709     862
1        2  113497    8844
2        3  113228   15602
3        4  114885   31357
4        5  113041   11862 
 Index(['movieId', 'imdbId', 'tmdbId'], dtype='object')


## Merge movies_train/test with MovieLens

In [5]:
train_merged = movies_train[['movieId']].merge(
    movielens_movies,
    on='movieId',
    how='left'
)

test_merged = movies_test[['movieId']].merge(
    movielens_movies,
    on='movieId',
    how='left'
)

print("Merged Train:\n", train_merged.head())
print("\nMerged Test:\n", test_merged.head())

Merged Train:
    movieId                             title          genres
0      619                         Ed (1996)          Comedy
1    33826                Saint Ralph (2004)    Comedy|Drama
2     1298       Pink Floyd: The Wall (1982)   Drama|Musical
3   140289              Men & Chicken (2015)    Comedy|Drama
4     3064  Poison Ivy: New Seduction (1997)  Drama|Thriller

Merged Test:
    movieId                                   title  \
0    45635       Notorious Bettie Page, The (2005)   
1     1373  Star Trek V: The Final Frontier (1989)   
2     7325                  Starsky & Hutch (2004)   
3      389              Colonel Chabert, Le (1994)   
4     8920                Country Girl, The (1954)   

                         genres  
0                         Drama  
1                 Action|Sci-Fi  
2  Action|Comedy|Crime|Thriller  
3             Drama|Romance|War  
4                         Drama  


## Merge with Links

In [6]:
train_complete = train_merged.merge(
    movielens_links,
    on='movieId',
    how='inner'
)

test_complete = test_merged.merge(
    movielens_links,
    on='movieId',
    how='inner'
)

print("Complete Train:\n", train_complete.head())
print("\nComplete Test:\n", test_complete.head())

Complete Train:
    movieId                             title          genres   imdbId  tmdbId
0      619                         Ed (1996)          Comedy   116165   32308
1    33826                Saint Ralph (2004)    Comedy|Drama   384488   25248
2     1298       Pink Floyd: The Wall (1982)   Drama|Musical    84503   12104
3   140289              Men & Chicken (2015)    Comedy|Drama  3877674  296313
4     3064  Poison Ivy: New Seduction (1997)  Drama|Thriller   119908   18222

Complete Test:
    movieId                                   title  \
0    45635       Notorious Bettie Page, The (2005)   
1     1373  Star Trek V: The Final Frontier (1989)   
2     7325                  Starsky & Hutch (2004)   
3      389              Colonel Chabert, Le (1994)   
4     8920                Country Girl, The (1954)   

                         genres  imdbId  tmdbId  
0                         Drama  404802   15402  
1                 Action|Sci-Fi   98382     172  
2  Action|Comedy|Crime|

## Extract years from titles

In [7]:
def extract_year(title: str) -> Optional[int]:
    if pd.isna(title):
        return None
    match = re.search(r'\((\d{4})\)', title)
    return int(match.group(1)) if match else None

train_complete['year'] = train_complete['title'].apply(extract_year).astype('Int64')
test_complete['year'] = test_complete['title'].apply(extract_year).astype('Int64')

### Address missing year values

In [8]:
train_missing = train_complete[train_complete['year'].isna()]
test_missing = test_complete[test_complete['year'].isna()]

print(f"Train missing years: {len(train_missing)}")
display(train_missing)

print(f"Test missing years: {len(test_missing)}")
display(test_missing)

Train missing years: 11


,movieId,title,genres,imdbId,tmdbId,year
832,156605,Paterson,(no genres listed),5247022,370755,<NA>
2482,171749,Death Note: Desu nôto (2006–2007),(no genres listed),877057,419787,<NA>
3745,171631,Maria Bamford: Old Baby,(no genres listed),6264596,455601,<NA>
3752,140956,Ready Player One,Action|Sci-Fi|Thriller,1677720,333339,<NA>
4269,167570,The OA,(no genres listed),4635282,432192,<NA>
4415,171495,Cosmos,(no genres listed),81846,409926,<NA>
4459,171891,Generation Iron 2,(no genres listed),6263642,447818,<NA>
4627,176601,Black Mirror,(no genres listed),2492564,452830,<NA>
5379,147250,The Adventures of Sherlock Holmes and Doctor W...,(no genres listed),229922,127605,<NA>
5819,143410,Hyena Road,(no genres listed),4034452,316042,<NA>


Test missing years: 1


,movieId,title,genres,imdbId,tmdbId,year
507,162414,Moonlight,Drama,4975722,376867,<NA>


In [9]:
load_dotenv()
TMDB_API_KEY = os.getenv('TMDB_API_KEY')
BASE_URL = 'https://api.themoviedb.org/3'

def get_year_from_tmdb(row) -> Optional[int]:
    if pd.notna(row['year']):
        return row['year']
    
    if pd.isna(row['tmdbId']):
        return None
    
    try:
        url = f"{BASE_URL}/movie/{int(row['tmdbId'])}"
        response = requests.get(url, params={'api_key': TMDB_API_KEY}, timeout=10)
        
        if response.status_code == 200:
            release_date = response.json().get('release_date', '')
            if release_date:
                time.sleep(0.25)
                return int(release_date.split('-')[0])
        
        time.sleep(0.25)
        return None
    except:
        return None

train_complete['year'] = train_complete.apply(get_year_from_tmdb, axis=1).astype('Int64') # type: ignore
test_complete['year'] = test_complete.apply(get_year_from_tmdb, axis=1).astype('Int64') # type: ignore

print("Missing years after TMDB lookup:", train_complete['year'].isna().sum(), test_complete['year'].isna().sum())

Missing years after TMDB lookup: 4 0


In [10]:
train_missing = train_complete[train_complete['year'].isna()]

print("Missing movies in training set after TMDB lookup:\n", train_missing)

Missing movies in training set after TMDB lookup:
       movieId                                              title  \
2482   171749                  Death Note: Desu nôto (2006–2007)   
4269   167570                                             The OA   
4415   171495                                             Cosmos   
5379   147250  The Adventures of Sherlock Holmes and Doctor W...   

                  genres   imdbId  tmdbId  year  
2482  (no genres listed)   877057  419787  <NA>  
4269  (no genres listed)  4635282  432192  <NA>  
4415  (no genres listed)    81846  409926  <NA>  
5379  (no genres listed)   229922  127605  <NA>  


After some investigation, I noticed that these four titles are series and not movies, so I would assume is safe to remove them.

In [11]:
train_complete = train_complete.dropna(subset=['year'])
test_complete = test_complete.dropna(subset=['year'])

print("Resulting Train Set:\n", train_complete.shape)
print("Resulting Test Set:\n", test_complete.shape)

Resulting Train Set:
 (6615, 6)
Resulting Test Set:
 (2835, 6)


### Address non specified genre values

In [12]:
train_no_genre = (train_complete['genres'] == '(no genres listed)').sum()
test_no_genre = (test_complete['genres'] == '(no genres listed)').sum()

print("Train non specified genres:", train_no_genre)
print("Test non specified genres:", test_no_genre)

Train non specified genres: 20
Test non specified genres: 9


Very little number, safe to delete.

In [13]:
train_complete = train_complete[train_complete['genres'] != '(no genres listed)']
test_complete = test_complete[test_complete['genres'] != '(no genres listed)']

print("Resulting Train Set:\n", train_complete.shape)
print("Resulting Test Set:\n", test_complete.shape)

Resulting Train Set:
 (6595, 6)
Resulting Test Set:
 (2826, 6)


### Address missing tmdbId

In [14]:
train_no_tmdbId = train_complete['tmdbId'].isna().sum()
test_no_tmdbId = test_complete['tmdbId'].isna().sum()

print("Train missing tmdbId:", train_no_tmdbId)
print("Test missing tmdbId:", test_no_tmdbId)

Train missing tmdbId: 4
Test missing tmdbId: 3


tmdbId is necessary for obtaining movie posters.

In [15]:
train_complete = train_complete.dropna(subset=['tmdbId'])
test_complete = test_complete.dropna(subset=['tmdbId'])

print("Final Train Set:\n", train_complete.shape)
print("Final Test Set:\n", test_complete.shape)

Final Train Set:
 (6591, 6)
Final Test Set:
 (2823, 6)


## Get poster urls

In [16]:
IMAGE_BASE_URL = 'https://image.tmdb.org/t/p/w500'

def get_poster_url(tmdbId: int) -> Optional[str]:
    try:
        url: str = f"{BASE_URL}/movie/{tmdbId}"
        time.sleep(0.02)
        response = requests.get(url, params={'api_key': TMDB_API_KEY}, timeout=10)

        if response.status_code == 200:
            poster_path = response.json().get('poster_path')
            if poster_path:
                return f"{IMAGE_BASE_URL}{poster_path}"
        
        return None
    except:
        return None

train_complete['poster_url'] = train_complete['tmdbId'].apply(get_poster_url)
test_complete['poster_url'] = test_complete['tmdbId'].apply(get_poster_url)

This urls will be later used to fetch the poster images for feature extraction.

In [21]:
train_missing_posters = train_complete['poster_url'].isna().sum()
test_missing_posters = test_complete['poster_url'].isna().sum()

print("Train missing posters:", train_missing_posters)
print("Test missing posters:", test_missing_posters)

Train missing posters: 54
Test missing posters: 23


Some poster links couldn't be fetched. These movies will be excluded because poster images are required for feature extraction.

In [26]:
train_complete = train_complete.dropna(subset=['poster_url'])
test_complete = test_complete.dropna(subset=['poster_url'])

print("Final Train Set:\n", train_complete.shape)
print("Final Test Set:\n", test_complete.shape)

Final Train Set:
 (6537, 7)
Final Test Set:
 (2800, 7)


## Final dataset

In [23]:
print("Final Train Set Sample:\n", train_complete.head())
print("Final Test Set Sample:\n", test_complete.head())



Final Train Set Sample:
    movieId                             title          genres   imdbId  tmdbId  \
0      619                         Ed (1996)          Comedy   116165   32308   
1    33826                Saint Ralph (2004)    Comedy|Drama   384488   25248   
2     1298       Pink Floyd: The Wall (1982)   Drama|Musical    84503   12104   
3   140289              Men & Chicken (2015)    Comedy|Drama  3877674  296313   
4     3064  Poison Ivy: New Seduction (1997)  Drama|Thriller   119908   18222   

   year                                         poster_url  
0  1996  https://image.tmdb.org/t/p/w500/oSNX3zyTpRMN0E...  
1  2004  https://image.tmdb.org/t/p/w500/3MdWgqUunIBWnc...  
2  1982  https://image.tmdb.org/t/p/w500/aElHyIdF5jmctF...  
3  2015  https://image.tmdb.org/t/p/w500/6lZ1EjOoWJiZaG...  
4  1997  https://image.tmdb.org/t/p/w500/l9Z43sz0qjTrmD...  
Final Test Set Sample:
    movieId                                   title  \
0    45635       Notorious Bettie Page, The 

In [24]:
print("=" * 60)
print("FINAL VERIFICATION")
print("=" * 60)

print(f"\nTRAIN_COMPLETE:")
print(f"  Total movies: {len(train_complete)}")
print(f"  Columns: {list(train_complete.columns)}")
print(f"  Missing title: {train_complete['title'].isna().sum()}")
print(f"  Missing genres: {train_complete['genres'].isna().sum()}")
print(f"  Missing year: {train_complete['year'].isna().sum()}")
print(f"  Missing tmdbId: {train_complete['tmdbId'].isna().sum()}")
print(f"  Missing poster_url: {train_complete['poster_url'].isna().sum()}")

print(f"\nTEST_COMPLETE:")
print(f"  Total movies: {len(test_complete)}")
print(f"  Missing title: {test_complete['title'].isna().sum()}")
print(f"  Missing genres: {test_complete['genres'].isna().sum()}")
print(f"  Missing year: {test_complete['year'].isna().sum()}")
print(f"  Missing tmdbId: {test_complete['tmdbId'].isna().sum()}")
print(f"  Missing poster_url: {test_complete['poster_url'].isna().sum()}")

FINAL VERIFICATION

TRAIN_COMPLETE:
  Total movies: 6537
  Columns: ['movieId', 'title', 'genres', 'imdbId', 'tmdbId', 'year', 'poster_url']
  Missing title: 0
  Missing genres: 0
  Missing year: 0
  Missing tmdbId: 0
  Missing poster_url: 0

TEST_COMPLETE:
  Total movies: 2800
  Missing title: 0
  Missing genres: 0
  Missing year: 0
  Missing tmdbId: 0
  Missing poster_url: 0


In [25]:
train_complete.to_csv('data/train_complete.csv', index=False)
test_complete.to_csv('data/test_complete.csv', index=False)

print("Cleaned datasets saved to 'data/train_complete.csv' and 'data/test_complete.csv'")

Cleaned datasets saved to 'data/train_complete.csv' and 'data/test_complete.csv'
